<a href="https://colab.research.google.com/github/grace3999/USV_Python/blob/colab/Notebooks/2_xr_Annotations_from_netcdf_PSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive containings required files: 1) csv of annotation features, 2) netcdf files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install netcdf4

In [0]:
import numpy as np
import pandas as pd
import os
import re
from scipy import stats
import xarray as xr

#visualizing results
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [0]:
#may need to be updated based on file naming scheme
def get_file_info(path, order):
    """takes in a file path for annotation selections table and finds the animal_number and session and saves each accordingly. 
    each file should be named with animal number and exp (e.g. 100_CPA.Table.1.selections)"""
    
    if order == 'animal':
      animal_number, session = re.split('_|-', path.split('/')[-1].split('.')[0])[0:2]
    else:
      session, animal_number = re.split('_|-', path.split('/')[-1].split('.')[0])[0:2] 
    
    print(animal_number, session)
    
    return animal_number, session

In [0]:
#need to be updated with animal naming scheme (e.g. int(animal_number) vs animal_number)
def create_annot_dataset(path, animal_number, session, annot_data, size_random):
    """takes in path of netcdf file (created using notebook 1) and corresponding annotations (created using notebook 0)
    and creates a new data set of only slices correspondingn to annotations (and random noise if set to True)"""
    
    #open xarray Dataset
    data = xr.open_dataset(path)
    print(data['slices'].shape)
    
    #get time_stamps of animal's annotations, select slices and save
    yes = annot_data[(annot_data['animal_number'] == int(animal_number)) & (annot_data['session'] ==  session)]['time_stamp'].sort_values().values
    data_yes = data.sel(slices = yes)
    
    #get slice indexs (e.g. time_stamps) that are not annotations, select slices and save
    slice_indexes = data['slices'].values
    no = np.setdiff1d(slice_indexes,yes)
    print(yes.shape[0] + no.shape[0])
    no_short = np.random.choice(no, size_random, replace=False)
    data_no = data.sel(slices = no_short)
    
    return data_yes, data_no

In [0]:
def compute_psd(Dataset):
  """takes in netcdf dataset and computes psd, saves as df"""
  
  psd_df = pd.DataFrame(data = Dataset.groupby('slices').sum(dim='times')['__xarray_dataarray_variable__'].values, index = Dataset['slices'].values, columns = Dataset['freq'].values/1000)
  
  return psd_df

In [0]:
def create_annotation_slice_psds(path, order, annot_data, size_random = 15):
    """takes in path of netcdf file (created using notebook 1), corresponding annotations (created using notebook 0),
    and size desired for random slicecs from each file.
    uses create_annot_dataset and compute_psd functions to create a new data set of computed features
    for only slices correspondingn to annotations and random noise"""

    animal_number, session = get_file_info(path, order)
    print(animal_number, session)
    
    #create datasets of slices of known annotations and a random selection of noise
    data_yes, data_no = create_annot_dataset(path, animal_number, session, annot_data, size_random)
    
    #compute psd
    yes_psd = compute_psd(data_yes)
    no_psd = compute_psd(data_no)

    #add psd to exisiting dataframe of known annotations
    annot_yes_psd = annot_data[(annot_data['animal_number'] ==  int(animal_number)) & (annot_data['session'] ==  session)].sort_values(by=['time_stamp'])
    annot_yes_psd = pd.merge(annot_yes_psd, yes_psd, left_on=annot_yes_psd['time_stamp'], right_on=yes_psd.index.values)
    
    #create and fill dataframe for randomly selected noise slices
    annot_no_psd = pd.DataFrame(columns = ['animal_number', 'exp', 'session', 'time_stamp', 'Annotation'], index = np.arange(0,size_random))
    annot_no_psd['animal_number'] = animal_number
    annot_no_psd['exp'] = annot_yes_psd['exp'].unique()[0]
    annot_no_psd['session'] = session
    annot_no_psd['Annotation'] = 'rand_noise'
    annot_no_psd['time_stamp'] = data_no['slices'].values
    annot_no_psd = pd.merge(annot_no_psd, no_psd, left_on=annot_no_psd['time_stamp'], right_on=no_psd.index.values)
    
    return annot_yes_psd, annot_no_psd 

Create data frame of annotation info

In [0]:
annot_path_cFos_CPA = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_CPApost_cFos_CPA.csv'
annot_path_cFos_neutral = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_neutral_cFos_neutral.csv'

annot_path_PETr1_CPA = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_CPApost_PETr1_CPA.csv'

annot_path_round2_CPA = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_CPApost_round2_CPA.csv'
annot_path_round2_neutral = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_neutral_round2_neutral.csv'

annot_paths = [annot_path_cFos_CPA, annot_path_cFos_neutral, annot_path_PETr1_CPA, annot_path_round2_CPA, annot_path_round2_neutral]

In [9]:
annot_data = pd.DataFrame()

for path in annot_paths:
    annot = pd.read_csv(path)
    annot = pd.DataFrame(data = annot)
    if 'radar' in annot['Annotation'].values:
      annot = annot[annot['Annotation'] != 'radar']
    if True in pd.isna(annot['Annotation']).values:
      annot['Annotation'] = ['BBC'] * annot['Annotation'].shape[0]
    
    print(annot.shape)
    print(annot.Annotation.value_counts())
    
    annot_data = annot_data.append(annot)

print(annot_data.shape)
annot_data.head()

(90, 7)
low           48
low multi     20
bbc           15
high           6
high multi     1
Name: Annotation, dtype: int64
(69, 7)
bbc           30
high          21
low           12
high multi     3
low multi      3
Name: Annotation, dtype: int64
(154, 7)
low            87
low multi      25
low complex    18
high           12
bbc            11
high multi      1
Name: Annotation, dtype: int64
(81, 7)
BBC    81
Name: Annotation, dtype: int64
(34, 7)
low            14
high            9
low complex     5
bbc             3
low multi       2
low             1
Name: Annotation, dtype: int64
(428, 7)


,Unnamed: 0,Annotation,High Freq (Hz),animal_number,exp,session,time_stamp
0,0,low,11243.4,534,cFos_CPA,CPApost,52357.5
1,1,low,11463.8,534,cFos_CPA,CPApost,56137.5
2,2,low,14329.8,534,cFos_CPA,CPApost,121095.0
3,3,low,11463.8,534,cFos_CPA,CPApost,121185.0
4,4,low,18298.1,534,cFos_CPA,CPApost,222300.0


In [0]:
netcdf_path_CPA = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/CPA'

netcdf_path_neutral = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/Neutral'

In [13]:
path = netcdf_path_CPA

path_names = []
files = os.listdir(path)
for file in files:
  path_names.append(path + "/" + file)

print(len(path_names))
path_names

26


['/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/CPA/533_CPApost_xr_Dataset.nc',
 '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/CPA/534_CPApost_xr_Dataset.nc',
 '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/CPA/535_CPApost_xr_Dataset.nc',
 '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/CPA/542_CPApost_xr_Dataset.nc',
 '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/CPA/543_CPApost_xr_Dataset.nc',
 '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/CPA/554_CPApost_xr_Dataset.nc',
 '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/CPA/555_CPApost_xr_Dataset.nc',
 '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/CPA/559_CPApost_xr_Dataset.nc',
 '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/CPA/648_CPApost_xr_Dataset.nc',
 '/content/gdrive/Shared drives/USV_eScience_Incubator/

In [14]:
### may have to update int(animal_number)

order = 'animal'
size_random = 25
session_name = 'CPA'
save_path = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/feature_data_frames'

annot_features_yes_psd = pd.DataFrame()
annot_features_no_psd = pd.DataFrame()

for path in path_names:
    
    annot_yes, annot_no = create_annotation_slice_psds(path, order, annot_data, size_random)
    
    annot_features_yes_psd = annot_features_yes_psd.append(annot_yes, ignore_index=True)
    annot_features_no_psd = annot_features_no_psd.append(annot_no, ignore_index=True)

#create and save combined dataframe of yes and no
annot_features_yes_psd.drop(['Unnamed: 0', 'High Freq (Hz)'], axis=1, inplace=True)
annot_features_full_psd = pd.concat([annot_features_yes_psd, annot_features_no_psd])
print(annot_features_full_psd.shape)
print(annot_features_full_psd.Annotation.value_counts())

annot_features_full_psd.to_csv(str(save_path + '/annot_psd_25noise_' + session_name + '.csv'))

533 CPApost
533 CPApost
(26666,)
26666
534 CPApost
534 CPApost
(26666,)
26666
535 CPApost
535 CPApost
(26666,)
26666
542 CPApost
542 CPApost
(26666,)
26666
543 CPApost
543 CPApost
(26666,)
26666
554 CPApost
554 CPApost
(26666,)
26666
555 CPApost
555 CPApost
(26666,)
26666
559 CPApost
559 CPApost
(26666,)
26666
648 CPApost
648 CPApost
(26666,)
26666
649 CPApost
649 CPApost
(26666,)
26666
650 CPApost
650 CPApost
(26666,)
26666
654 CPApost
654 CPApost
(26666,)
26666
655 CPApost
655 CPApost
(26666,)
26666
656 CPApost
656 CPApost
(26666,)
26666
621 CPApost
621 CPApost
(26666,)
26666
632 CPApost
632 CPApost
(26666,)
26666
629 CPApost
629 CPApost
(26666,)
26666
625 CPApost
625 CPApost
(26666,)
26666
624 CPApost
624 CPApost
(26666,)
26666
626 CPApost
626 CPApost
(26666,)
26666
622 CPApost
622 CPApost
(26666,)
26666
630 CPApost
630 CPApost
(26666,)
26666
623 CPApost
623 CPApost
(26666,)
26666
627 CPApost
627 CPApost
(26666,)
26666
631 CPApost
631 CPApost
(26666,)
26666
628 CPApost
628 CPApost
(